In [ ]:
!pip install ultralytics
import torch
from torchvision import transforms
from PIL import Image
import cv2
from ultralytics import YOLO
import numpy as np
from collections import Counter
import glob


In [ ]:
yolo_model = YOLO("/content/yolo_best.pt")


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
cnn_model = models.resnet18(pretrained=False)
cnn_model.fc = nn.Linear(cnn_model.fc.in_features, 3)
state_dict = torch.load("/content/cnn_best_epoch5.pt", map_location='cpu')
cnn_model.load_state_dict(state_dict)

cnn_model.eval()
print("CNN model loaded successfully")


In [ ]:
from google.colab import files

uploaded = files.upload()
test_img_path = list(uploaded.keys())[0]
print("Uploaded:", test_img_path)


In [ ]:
results = yolo_model.predict(source=test_img_path, conf=0.05, save=True, save_txt=True)
print("YOLO done!")


In [ ]:
label_file = glob.glob("runs/detect/predict*/labels/*.txt")[0]

with open(label_file) as f:
    cls = [int(l.split()[0]) for l in f.readlines()]

count = Counter(cls)
print("Normal:", count[0])
print("Reversal:", count[1])
print("Corrected:", count[2])

class_order = ["Normal", "Reversal", "Corrected"]
dominant = class_order[np.argmax([count[i] for i in range(3)])]
print("YOLO Dominant handwriting type =", dominant)


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

img = Image.open(test_img_path).convert("RGB")
input_tensor = transform(img).unsqueeze(0)

pred = cnn_model(input_tensor)
cls_idx = pred.argmax().item()
class_map = {0:"Normal",1:"Reversal",2:"Corrected"}

print("CNN Prediction =", class_map[cls_idx])


In [ ]:
def to_dataset_style(in_path, out_path="/content/test_clean.png"):
    img = cv2.imread(in_path)
    g   = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # If background is white-ish, invert to white-on-black
    if np.mean(g) > 127:
        g = 255 - g

    # Hard binarize + slight thicken strokes
    _, bw = cv2.threshold(g, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    bw = cv2.GaussianBlur(bw, (3,3), 0)
    _, bw = cv2.threshold(bw, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    kernel = np.ones((2,2), np.uint8)
    bw = cv2.dilate(bw, kernel, iterations=1)

    # Put on black square canvas (like training)
    h, w = bw.shape
    s = max(h, w)
    canvas = np.zeros((s, s), dtype=np.uint8)
    y0 = (s - h)//2; x0 = (s - w)//2
    canvas[y0:y0+h, x0:x0+w] = bw

    cv2.imwrite(out_path, canvas)
    return out_path


In [ ]:
clean_path = to_dataset_style(test_img_path)

In [ ]:
from ultralytics import YOLO
from collections import defaultdict
import numpy as np

yolo_model = YOLO("/content/yolo_best.pt")
res = yolo_model.predict(source=clean_path, conf=0.35, iou=0.5, verbose=False)[0]

weights = defaultdict(float)
counts  = defaultdict(int)

cls_ids = res.boxes.cls.cpu().numpy().astype(int)
confs   = res.boxes.conf.cpu().numpy().astype(float)

for c, cf in zip(cls_ids, confs):
    counts[c]  += 1
    weights[c] += cf

name_map = {0:"Normal", 1:"Reversal", 2:"Corrected"}

totals = {name_map[k]: counts.get(k, 0)  for k in [0,1,2]}
scores = {name_map[k]: weights.get(k, 0.) for k in [0,1,2]}
print("Raw counts:", totals)
print("Weighted scores:", {k: round(v,2) for k,v in scores.items()})
ordered = sorted(scores.items(), key=lambda x: x[1], reverse=True)
top, second = ordered[0][1], ordered[1][1]
margin = top - second

if totals["Normal"] >= totals["Reversal"] and totals["Normal"] >= totals["Corrected"]:
    if margin < 0.20:
        yolo_majority = "Normal"
    else:
      yolo_majority = ordered[0][0]

print("YOLO decision:", yolo_majority)


In [ ]:
t = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

img = Image.open(clean_path).convert("RGB")
logits = cnn_model(t(img).unsqueeze(0))
probs = torch.softmax(logits, dim=1).detach().cpu().numpy()[0]
classes = ["Normal","Reversal","Corrected"]
cnn_top = classes[int(np.argmax(probs))]
margin = np.sort(probs)[-1] - np.sort(probs)[-2]
cnn_decision = "Mixed" if margin < 0.12 else cnn_top

print("CNN probs:", {c: round(float(p),2) for c,p in zip(classes, probs)})
print("CNN decision:", cnn_decision)


In [ ]:
print("\nFINAL SUMMARY:")
print("YOLO says →", yolo_majority)
print("CNN says →", class_map[cls_idx])


In [ ]:
if yolo_majority != "Mixed":
    final_decision = yolo_majority + " (YOLO letter pattern dominant)"
else:
    final_decision = cnn_decision + " (CNN fallback due to local ambiguity)"

print("FINAL DECISION:", final_decision)
